In [1]:
import gzip
import json
import csv
import pandas as pd
import numpy as np
import re

from pandas import json_normalize

import my_functions
%load_ext autoreload
%autoreload 2

Se realiza la extracción y carga del archivo original, 'steam_games.json.gz'

In [2]:
# Ruta al archivo comprimido .json.gz
ruta_archivo_comprimido = 'Datasets/Datasets Originales/steam_games.json.gz'
ruta_archivo_csv = 'Datasets/Datasets ETL/steam_games_raw.csv'

# Lista para almacenar los datos
datos_json = []

# Descomprimir el archivo y cargar cada línea como un objeto JSON
with gzip.open(ruta_archivo_comprimido, 'rt', encoding='utf-8') as archivo_comprimido:
    for linea in archivo_comprimido:
        try:
            # Cargar cada línea como un objeto JSON
            datos_json.append(json.loads(linea))
            
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en línea: {e}")

# Extraer las claves (nombres de columnas) de los primeros datos (puedes ajustar el índice)
columnas = list(datos_json[0].keys())

# Escribir los datos en un archivo CSV
with open(ruta_archivo_csv, 'w', newline='', encoding='utf-8') as archivo_csv:
    escritor_csv = csv.DictWriter(archivo_csv, fieldnames=columnas)
    
    # Escribir las cabeceras
    escritor_csv.writeheader()
    
    # Escribir los datos
    escritor_csv.writerows(datos_json)

print(f"Archivo CSV '{ruta_archivo_csv}' creado con éxito.")

Archivo CSV 'Datasets/Datasets ETL/steam_games_raw.csv' creado con éxito.


Se realiza la carga desde el archivo steam_games_raw.csv a un Dataframe

In [3]:
# Ruta al archivo CSV
ruta_archivo_csv = 'Datasets/Datasets ETL/steam_games_raw.csv'

# Cargar el archivo CSV en un DataFrame
df_games = pd.read_csv(ruta_archivo_csv)

C:\Users\matia\AppData\Local\Temp\ipykernel_14792\3515485724.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_games = pd.read_csv(ruta_archivo_csv)


In [4]:
my_functions.describe_df(df_games)

Cantidad Registros:  120445
Cantidad Campos:  13


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,publisher,object,96381,80.02,24064,19.98,8236,6.84
1,genres,object,91593,76.05,28852,23.95,883,0.73
2,app_name,object,88312,73.32,32133,26.68,32094,26.65
3,title,object,90360,75.02,30085,24.98,30054,24.95
4,url,object,88310,73.32,32135,26.68,32135,26.68
5,release_date,object,90377,75.04,30068,24.96,3582,2.97
6,tags,object,88473,73.46,31972,26.54,15395,12.78
7,reviews_url,object,88312,73.32,32133,26.68,32132,26.68
8,specs,object,88980,73.88,31465,26.12,4649,3.86
9,price,object,89687,74.46,30758,25.54,162,0.13


Se observa:
* Registros iniciales: 120445 
* Campos: 13, de tipo object (12) y float64 (1)
* Hay una cantidad considerable de nulos en todas las columnas.

Se eliminan, en caso de haberlos, espacios vacíos de más en las cadenas de texto para facilitar el procesamiento posterior.<br>
También se reemplazan, en caso de existir, valores de cadenas vacías en todos los campos por nulos.

In [5]:
# Se eliminan espacios vacíos de más
df_games = df_games.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Reemplazar espacios vacíos por NaN
df_games.replace(r'^\s*$', np.nan, regex=True, inplace=True)

C:\Users\matia\AppData\Local\Temp\ipykernel_14792\348993469.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_games = df_games.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Se realiza una visualización de algunos registros:

In [6]:
df_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se observan registros que únicamente contienen valores nulos. <br> 
Se procede a eliminarlos.



In [7]:
df_games.dropna(how='all', inplace=True)

Se visualizan nuevamente algunos registros:

In [8]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"['Single-player', 'Steam Achievements']",1.99,False,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,658870.0,"xropi,stev3ns"


Se convierte el campo 'id' (inicialmente de tipo float) a tipo string dado que es una variable categórica.

In [9]:
# Modificar tipo de columna 'id' a string
df_games['id'] = df_games['id'].astype(str)

# Eliminar el punto y lo que sigue con una cadena vacía en la columna 'id'
df_games['id'] = df_games['id'].str.replace(r'\..*', '', regex=True)

In [10]:
# Se verifican las modificaciones
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro


In [11]:
# Se verifican las modificaciones
my_functions.describe_df(df_games)

Cantidad Registros:  32135
Cantidad Campos:  13


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,publisher,object,8071,25.12,24064,74.88,8236,25.63
1,genres,object,3283,10.22,28852,89.78,883,2.75
2,app_name,object,2,0.01,32133,99.99,32094,99.87
3,title,object,2050,6.38,30085,93.62,30054,93.52
4,url,object,0,0.00,32135,100.00,32135,100.00
5,release_date,object,2067,6.43,30068,93.57,3582,11.15
6,tags,object,163,0.51,31972,99.49,15395,47.91
7,reviews_url,object,2,0.01,32133,99.99,32132,99.99
8,specs,object,670,2.08,31465,97.92,4649,14.47
9,price,object,1377,4.29,30758,95.71,162,0.50


Se buscan filas duplicadas, sin encontrar resultados.

In [12]:
df_games[df_games.duplicated()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


## Campo 'id'

Se buscan 'id' duplicados. Este paso es de suma importancia dado que en este dataset deben contenerse registros únicos para cada juego.

In [13]:
# 'Imprimir filas con 'id' duplicados'
print(f"Cantidad total de filas duplicadas: {len(df_games[df_games.duplicated(subset='id', keep=False)])}")

df_games[df_games.duplicated(subset='id', keep=False)]

Cantidad total de filas duplicadas: 4


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,nan,NaN
102204,Bethesda Softworks,['Action'],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"['Action', 'FPS', 'Gore', 'Violent', 'Alternat...",http://steamcommunity.com/app/612880/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",59.99,False,612880,Machine Games
102883,Bethesda Softworks,['Action'],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"['Action', 'FPS', 'Gore', 'Violent', 'Alternat...",http://steamcommunity.com/app/612880/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",59.99,False,612880,Machine Games
119271,"Warner Bros. Interactive Entertainment, Feral ...","['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...",NaN,"['Single-player', 'Steam Achievements', 'Steam...",19.99,False,nan,"Rocksteady Studios,Feral Interactive (Mac)"


Se observa:<br>
* 2 registros duplicados para el 'id' 612880.<br>
Se procede a eliminar el segundo.

* 2 id con valor 'nan':<br>
    * El primero solo contiene datos que no resultan relevantes en el campo 'url', por lo que se procede a eliminar ese registro.
    * El segundo contiene datos relevantes, por lo que se procede a explorar este registro para determinar cómo tratarlo.

In [14]:
df_games = df_games.drop(102883)

In [15]:
df_games.loc[88384]['url']

'http://store.steampowered.com/'

In [16]:
df_games = df_games.drop(88384)

Al explorar el registro con id 'nan', que contiene datos relevantes, se identifica que está duplicado en otro registro que sí cuenta con un id.<br>
Se elimina el registro con id 'nan'.

In [17]:
df_games[df_games['app_name'] == 'Batman: Arkham City - Game of the Year Edition']

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...",http://steamcommunity.com/app/200260/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...",NaN,"['Single-player', 'Steam Achievements', 'Steam...",19.99,False,nan,"Rocksteady Studios,Feral Interactive (Mac)"


In [18]:
df_games = df_games.drop(119271)

Se verifica que no hayan id duplicados.

In [19]:
df_games[df_games.duplicated(subset='id', keep=False)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


Habiendo eliminado los registros duplicados de 'id' los valores correspondientes pueden ser utilizados como identificador único del contenido de cada juego.

Se renombra la columna 'id' por 'game_id'.

Como se vera en la exploración de información sobre reseñas e items de usuarios (puntos 1.2 y 1.3 del ETL) este campo es compartido entre las 3 tablas, por lo que se le aplicará 'game_id' como denominación en común para este campo en los otros datasets.

In [20]:
# Cambiar el nombre de la columna 'url' a 'game_url'
df_games.rename(columns={'id': 'game_id'}, inplace=True)

## Campos 'app_name' y 'title'

In [21]:
# Se buscan registros donde los 2 campos sean nulos
df_games[df_games['app_name'].isnull() & df_games['title'].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,game_id,developer
90890,NaN,"['Action', 'Indie']",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"['Action', 'Indie']",http://steamcommunity.com/app/317160/reviews/?...,"['Single-player', 'Game demo']",NaN,False,317160,NaN


In [22]:
# Contar la cantidad total de registros
cantidad_total_registros = len(df_games)

# Imprimir la cantidad total de registros
print(f"Cantidad total de registros: {cantidad_total_registros}")

# Cantidad de registros para cada campo
app_name_vacios = df_games[df_games['app_name'].isna()]
title_vacios = df_games[df_games['title'].isna()]

print(f"Cantidad de registros con 'app_name' vacío: {len(app_name_vacios)}")
print(f"Cantidad de registros con 'title' vacío: {len(title_vacios)}")

# Contar cantidad de registros con los 3 campos llenos
registross_clave_llenos = df_games[~((df_games['app_name'].isna()) | 
                                   (df_games['title'].isna()))]

print(f"Cantidad de registros los 3 campos campos llenos: {len(registross_clave_llenos)}")

# Contar cantidad de registros con al menos uno de los campos vacío
registros_uno_vacio = df_games[((df_games['app_name'].isna()) |  
                                (df_games['title'].isna()))]

print(f"Cantidad de registros con al menos un campo vacío: {len(registros_uno_vacio)}")

Cantidad total de registros: 32132
Cantidad de registros con 'app_name' vacío: 1
Cantidad de registros con 'title' vacío: 2049
Cantidad de registros los 3 campos campos llenos: 30083
Cantidad de registros con al menos un campo vacío: 2049


In [23]:
# Se explora registro con 'app_name' vacío
app_name_vacios

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,game_id,developer
90890,NaN,"['Action', 'Indie']",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"['Action', 'Indie']",http://steamcommunity.com/app/317160/reviews/?...,"['Single-player', 'Game demo']",NaN,False,317160,NaN


Se explora el registro con 'app_name' vacío (id 317160). 
Siguiendo el link provisto en el campo 'url' conduce a https://store.steampowered.com/app/292600/Duet/, identificando que se redirige a la app con id 292600. 
Siendo que estos datos son redundantes, se elimina este registro.

In [24]:
# Eliminar filas donde 'game_id' es igual a 317160
df_games = df_games[df_games['game_id'] != '317160']

# Se verifica eliminación
app_name_vacios = df_games[df_games['app_name'].isna()]
print(f"Cantidad de registros con 'app_name' vacío: {len(app_name_vacios)}")

Cantidad de registros con 'app_name' vacío: 0


Se exploran registros únicos de los campos 'app_name' y 'title'.

In [25]:
# Cantidad total de registros
cantidad_total_registros = len(df_games)
print(f"Cantidad total de registros: {cantidad_total_registros}")

# Registros únicos de app_name
app_name_unicos = df_games['app_name'].nunique()
print(f"Registros únicos de 'app_name': {app_name_unicos}")

# Registros únicos de title
title_unicos = df_games['title'].nunique()
print(f"Registros únicos de 'title': {title_unicos}")

Cantidad total de registros: 32131
Registros únicos de 'app_name': 32094
Registros únicos de 'title': 30054


Se observan registros con 'app_name' y 'title' repetidos.

Tras realizar una exploración de estos registros a partir de su 'url' (que al igual que 'game_id' es única para todos los registros) no se ecuentran juegos duplicados.

In [26]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')

duplicates = my_functions.duplicates(df_games, 'app_name')
duplicates

Cantidad de valores duplicados para app_name : 36
Valores duplicados: 
 ['AirMech® Soundtrack' 'Aliens: Colonial Marines - Reconnaissance Pack'
 'Alter Ego' 'Altitude' 'Archangel' 'Black Rose' 'Borealis' 'Castles'
 'Colony' 'Dark Matter' 'Escape Room' 'Goro' 'Hero' 'Invasion'
 'Invisible Apartment 2' 'Invisible Apartment Zero' 'Luna'
 'MORE SWEATER? OK!' 'Mars 2030' 'New York Bus Simulator' 'Parasite'
 'Prism' 'Puzzle Blocks' 'Rumpus' 'See No Evil' 'Slice&Dice' 'Solitaire'
 'Soundtrack' 'Streamline' 'Taxi' 'The Dream Machine: Chapter 4' 'The Fog'
 'Total Extreme Wrestling' 'Ultimate Arena' 'WARZONE' 'Warhammer Quest']


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,game_id,developer
107158,NaN,NaN,AirMech® Soundtrack,NaN,http://store.steampowered.com/app/599520/AirMe...,NaN,"['Strategy', 'Action', 'Indie']",http://steamcommunity.com/app/599520/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",9.99,False,599520,NaN
118857,Carbon Games,"['Action', 'Free to Play', 'Indie', 'Strategy']",AirMech® Soundtrack,AirMech® Soundtrack,http://store.steampowered.com/app/216173/AirMe...,2012-11-13,"['Strategy', 'Action', 'Free to Play', 'Indie']",http://steamcommunity.com/app/216173/reviews/?...,"['Single-player', 'Multi-player', 'Co-op', 'Do...",9.99,False,216173,Carbon Games
118489,SEGA,['Action'],Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/219441/Alien...,2013-05-07,['Action'],http://steamcommunity.com/app/219441/reviews/?...,"['Single-player', 'Multi-player', 'Co-op', 'Do...",29.99,False,219441,Gearbox Software
118488,SEGA,['Action'],Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/224850/Alien...,2013-05-07,['Action'],http://steamcommunity.com/app/224850/reviews/?...,"['Single-player', 'Multi-player', 'Co-op', 'Do...",NaN,False,224850,Gearbox Software
119713,Viva Media,['Adventure'],Alter Ego,Alter Ego,http://store.steampowered.com/app/63110/Alter_...,2010-08-03,"['Adventure', 'Point & Click', 'Mystery', 'Det...",http://steamcommunity.com/app/63110/reviews/?b...,['Single-player'],9.99,False,63110,bitComposer Games
99223,Choose Multiple LLC,"['Adventure', 'Casual', 'Indie', 'RPG', 'Simul...",Alter Ego,Alter Ego,http://store.steampowered.com/app/664780/Alter...,1986-05-01,"['RPG', 'Casual', 'Indie', 'Adventure', 'Simul...",http://steamcommunity.com/app/664780/reviews/?...,"['Single-player', 'Captions available', 'Steam...",7.99,False,664780,Choose Multiple LLC
106506,NaN,NaN,Altitude,Altitude,http://store.steampowered.com/app/620010/Altit...,2017-04-14,"['Movie', 'Detective', 'Action', 'Assassin']",http://steamcommunity.com/app/620010/reviews/?...,['Captions available'],4.99,False,620010,NaN
119852,Nimbly Games,"['Action', 'Free to Play', 'Indie']",Altitude,Altitude,http://store.steampowered.com/app/41300/Altitude/,2009-12-04,"['Free to Play', 'Action', 'Indie', 'Multiplay...",http://steamcommunity.com/app/41300/reviews/?b...,"['Single-player', 'Multi-player', 'Cross-Platf...",Free,False,41300,Nimbly Games
98879,"Juan Gabriel Gomila, Benjamin Miró",['RPG'],Archangel,Archangel,http://store.steampowered.com/app/661520/Archa...,2017-07-01,"['RPG', 'RPGMaker', 'JRPG', 'Action RPG']",http://steamcommunity.com/app/661520/reviews/?...,['Single-player'],1.99,False,661520,Frogames
104765,NaN,NaN,Archangel,NaN,http://store.steampowered.com/app/553880/Archa...,NaN,"['Action', 'Violent', 'Simulation', 'VR', 'Mec...",http://steamcommunity.com/app/553880/reviews/?...,"['Single-player', 'HTC Vive', 'Oculus Rift', '...",29.99,False,553880,NaN


Siguiendo las url se puede observar que más allá de compartir el 'app_name', estos registros difieren en varios campos y no se tratan del mismo contenido.

Por otro lado, considerando que todos tienen 'game_id' único como identificador principal, se define no realizar modificaciones.

Abajo se puede observar el codigo utilizado para explorar las url de los 'app_name' duplicados. Se puede realizar la exploración cambiando el valor de la variable 'app_name' por alguno de los valores duplicados (en este ejemplo app_name = 'AirMech® Soundtrack').

In [27]:
'''
Valores únicos de 'app_name':

 'AirMech® Soundtrack' 'Aliens: Colonial Marines - Reconnaissance Pack'
 'Alter Ego' 'Altitude' 'Archangel' 'Black Rose' 'Borealis' 'Castles'
 'Colony' 'Dark Matter' 'Escape Room' 'Goro' 'Hero' 'Invasion'
 'Invisible Apartment 2' 'Invisible Apartment Zero' 'Luna'
 'MORE SWEATER? OK!' 'Mars 2030' 'New York Bus Simulator' 'Parasite'
 'Prism' 'Puzzle Blocks' 'Rumpus' 'See No Evil' 'Slice&Dice' 'Solitaire'
 'Soundtrack' 'Streamline' 'Taxi' 'The Dream Machine: Chapter 4' 'The Fog'
 'Total Extreme Wrestling' 'Ultimate Arena' 'WARZONE' 'Warhammer Quest'
'''

app_name = 'AirMech® Soundtrack'

duplicates_app_name = duplicates[duplicates['app_name'] == app_name]

print("\n\nDuplicados de ", app_name, "\n\n", duplicates_app_name['url'], "\n\n")
duplicates_app_name



Duplicados de  AirMech® Soundtrack 

 107158    http://store.steampowered.com/app/599520/AirMe...
118857    http://store.steampowered.com/app/216173/AirMe...
Name: url, dtype: object 




,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,game_id,developer
107158,NaN,NaN,AirMech® Soundtrack,NaN,http://store.steampowered.com/app/599520/AirMe...,NaN,"['Strategy', 'Action', 'Indie']",http://steamcommunity.com/app/599520/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",9.99,False,599520,NaN
118857,Carbon Games,"['Action', 'Free to Play', 'Indie', 'Strategy']",AirMech® Soundtrack,AirMech® Soundtrack,http://store.steampowered.com/app/216173/AirMe...,2012-11-13,"['Strategy', 'Action', 'Free to Play', 'Indie']",http://steamcommunity.com/app/216173/reviews/?...,"['Single-player', 'Multi-player', 'Co-op', 'Do...",9.99,False,216173,Carbon Games


In [28]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

## Eliminación de campos no relevantes

A partir de esta exploración inicial se pueden identificar campos que no serán de utilidad en relación a los objetivos propuestos en el presente trabajo, por lo cual se decide eliminarlos. Dichos campos son: 'publisher', 'title', 'url', 'tags', 'reviews_url', 'specs', 'price', 'early_access'.

El campo 'title', si bien es importante, contiene la misma información o muy similar al campo 'app_name' (que no contiene ningún valor vacío). Por ello se determina eliminarla y usar la información de 'app_name'.

El campo 'url' cuenta con registros únicos para cada 'game_id'. Es decir, el link a la página web de ese juego en Steam.
Esta información podría resultar útil en caso de requerir acceso a algún tipo de información sobre los juegos.
Sin embargo, a los efecto del presente trabajo no será necesaria y en todo caso puede ser recuperada fácilmente.

La información relativa al resto de los campos que se define eliminar se puede encontrar en el diccionario de datos adjunto al proyecto. 
Como se podrá observar, dichos campos no son relevantes para los objetivos del presente.

In [29]:
# Remover columnas del DataFrame df_games
columnas_a_eliminar_games = ['publisher', 'title', 'url', 'tags', 'reviews_url', 'specs', 'price', 'early_access']
df_games = df_games.drop(columns=columnas_a_eliminar_games)

Hasta aquí hemos seleccionado los campos necesarios para el proyecto. 

Se procede a explorar el resto de los campos, a saber: 
genres, release_date, developer.

In [30]:
# Se verifican los cambios realizados
my_functions.describe_df(df_games)

Cantidad Registros:  32131
Cantidad Campos:  5


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,genres,object,3282,10.21,28849,89.79,883,2.75
1,app_name,object,0,0.00,32131,100.00,32094,99.88
2,release_date,object,2066,6.43,30065,93.57,3582,11.15
3,game_id,object,0,0.00,32131,100.00,32131,100.00
4,developer,object,3297,10.26,28834,89.74,10992,34.21


## Campo 'release_date'

Este campo es de tipo cadena de texto. Se explorará con el objetivo de convertirlo a formato de fecha, particularmente nos insteresa el año de lanzamiento del juego.

In [31]:
duplicates = my_functions.duplicates(df_games, 'release_date')
duplicates

Cantidad de valores duplicados para release_date : 2207
Valores duplicados: 
 ['1981-01-01' '1983-01-01' '1987-01-01' ... 'Winter 2018' 'coming soon'
 nan]


,genres,app_name,release_date,game_id,developer
107426,NaN,Gallagher: Mad As Hell,1981-01-01,579680,NaN
97556,NaN,Gallagher: Two Real,1981-01-01,614510,NaN
107429,NaN,Gallagher: The Maddest,1983-01-01,579630,NaN
107427,NaN,Gallagher: Stuck in the 60's,1983-01-01,593470,NaN
116582,"['Adventure', 'Simulation']",Sid Meier's Pirates! Gold Plus (Classic),1987-01-01,327380,"MicroProse Software, Inc"
...,...,...,...,...,...
120387,NaN,SpaceWalker,NaN,705860,NaN
120394,"['Action', 'Casual', 'Indie']",BOMBFEST,NaN,729640,The BOMBFEST Team
120395,NaN,LIV Client,NaN,755540,NaN
120398,"['Action', 'Adventure', 'Indie', 'Early Access']",RECHARGE COMPLETE,NaN,708070,rechargecomplete


Pueden observarse valores con un formato simil fecha (por ejemplo, '1981-01-01'), otros como texto que puede informar sobre la fecha de lanzamiento (por ejemplo, 'Winter 2018' 'coming soon') así como valores nulos.

Se procede a trabajar sobre el campo 'release_date' para extraer el año de lanzamiento de aquellos valores que tienen un formato que puede ser interpretado como una fecha en un nuevo campo, 
'release_year', y se eliminará 'release_date'.

Para aquellos valores de 'release_date' que no contengan información para extraer el año, se consigna como valor '0', que se puede interpretar como "sin información".

In [32]:
# Crear una función personalizada para extraer el año o devolver 0 si no se puede extraer
def extract_year_or_zero(posted_text):
    try:
        year = int(re.search(r'(\d{4})', posted_text).group(1))
        return year
    except (AttributeError, TypeError):
        return 0
    
# Aplicar la función a la columna 'release' y crear la columna 'release_year'
df_games['release_year'] = df_games['release_date'].apply(extract_year_or_zero)

# Remover columna 'release_date'
df_games = df_games.drop(columns=['release_date'])

In [33]:
# Se verifican modificaciones
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32131 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28849 non-null  object
 1   app_name      32131 non-null  object
 2   game_id       32131 non-null  object
 3   developer     28834 non-null  object
 4   release_year  32131 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


Se explora el nuevo campo, 'release_year'.

Como se puede observar el tipo de datos es entero (int64).

Por otro lado, la función aplicada para extraer el año parece haber funcionado, devolviendo valores entre 1970 y 2019, además de 0 para los registros de 'release_date' que no se pudieron convertir al formato deseado.

In [34]:
my_functions.duplicates(df_games, 'release_year')

Cantidad de valores duplicados para release_year : 40
Valores duplicados: 
 [   0 1970 1981 1982 1983 1984 1985 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007
 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]


,genres,app_name,game_id,developer,release_year
120444,NaN,Maze Run VR,681550,NaN,0
108361,NaN,AmaranTime,566800,NaN,0
102942,NaN,Bits n Bullets,730180,NaN,0
108322,NaN,Merry Snowballs,567730,NaN,0
102951,NaN,SONAR - A Virtual Reality Experience,738790,NaN,0
...,...,...,...,...,...
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,761140,Kotoshiro,2018
102742,"['Action', 'Adventure', 'Indie', 'RPG', 'Strat...",The Legendary Player - Make Your Reputation - ...,699570,Ninetales Studios,2019
102638,"['Adventure', 'Free to Play', 'Indie']",K'NOSSOS,696770,Svarun Entertainment,2019
102531,"['Action', 'Adventure', 'Indie']",Raji: An Ancient Epic,730390,Nodding Heads Games,2019


## Campo 'genres'

Se procede con la exploración.

Si bien los valores son de tipo string, se observa que el texto se asemeja a una lista que contiene como elementos distintos valores de géneros. Es decir, cada 'game_id' cuenta con una lista de los géneros que le corresponde.

Se encuentran pocos duplicados, lo cual indica que muchos juegos cuentan con los mismos géneros.

In [35]:
my_functions.duplicates(df_games, 'genres')

Cantidad de valores duplicados para genres : 581
Valores duplicados: 
 ["['Accounting']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie', 'Early Access']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie', 'Massively Multiplayer', 'RPG', 'Simulation']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie', 'RPG', 'Strategy']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie', 'RPG']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie', 'Simulation', 'Sports', 'Strategy']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Indie']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'Massively Multiplayer', 'RPG']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'RPG', 'Strategy']"
 "['Action', 'Adventure', 'Casual', 'Free to Play', 'RPG']"
 "['Action', 'Adventure', 'Casual', 'Indie', 'Early Access']"
 "['Action', 'Adventure', 'Casual', 'Indie', 'Massively Multiplayer', 'Racing', 'RPG', 'Simulation', 'Sports', 'Strategy']"
 "['Action',

,genres,app_name,game_id,developer,release_year
93529,['Accounting'],Quicken Legal Business Pro,411350,Nolo,2015
105502,['Accounting'],Professor Teaches QuickBooks 2017,623590,Individual Software,2017
112697,['Accounting'],Quicken WillMaker Plus 2016,411340,Nolo,2015
100606,['Accounting'],Quicken WillMaker Plus 2018,732710,Nolo,2017
96809,['Accounting'],Quicken WillMaker Plus 2017,555810,Nolo,2016
...,...,...,...,...,...
120428,NaN,Robotpencil Presents: Exercise: Brushwork,775640,NaN,2018
120429,NaN,Robotpencil Presents: Creative Composition,777930,NaN,2018
120430,NaN,The Gamble House,775370,NaN,2016
120431,NaN,Kalen Chock Presents: 2017 Free Tutorial,777950,NaN,2018


Se procederá a extraer los valores dados como campos independientes, es decir, que cada género contará con una columa. Los valores para cada género serán 0 si ese juego no es del género dado, 1 en caso que sí lo sea.

A las columnas de los géneros se les inserta en su nombre el prefijo 'gnr_' antes del nombre de dicho género, de forma que facilite ciertas operaciones para trabajar con estos campos en su conjunto.

Habiendo realizado esto se puede eliminar la columna 'genres'.

In [36]:
# Eliminar corchetes y comillas simples de la columna 'genres'
df_games['genres'] = df_games['genres'].str.replace("[", "").str.replace("]", "").str.replace("'", "")

# Dividir la columna 'genres' en columnas binarias con el prefijo 'gnr_'
df_games = pd.concat([df_games, df_games['genres'].str.get_dummies(', ').add_prefix('gnr_')], axis=1)

# Remover columna 'genres'
df_games = df_games.drop(columns=['genres'])

# Se verifican las modificaciones
df_games.columns

Index(['app_name', 'game_id', 'developer', 'release_year', 'gnr_Accounting',
       'gnr_Action', 'gnr_Adventure', 'gnr_Animation &amp; Modeling',
       'gnr_Audio Production', 'gnr_Casual', 'gnr_Design &amp; Illustration',
       'gnr_Early Access', 'gnr_Education', 'gnr_Free to Play', 'gnr_Indie',
       'gnr_Massively Multiplayer', 'gnr_Photo Editing', 'gnr_RPG',
       'gnr_Racing', 'gnr_Simulation', 'gnr_Software Training', 'gnr_Sports',
       'gnr_Strategy', 'gnr_Utilities', 'gnr_Video Production',
       'gnr_Web Publishing'],
      dtype='object')

Como se puede ver ahora contamos con los campos 'app_name', 'game_id', 'developer', 'release_year' y varios campos para los géneros.

Se identifica un detalle en el formato de texto de algunos nombres de géneros y se corrijen.

In [37]:
# Cambiar el nombre de las columna (se reemplaza '&amp' por '&')
df_games.rename(columns={'gnr_Animation &amp; Modeling': 'gnr_Animation & Modeling'}, inplace=True)
df_games.rename(columns={'gnr_Design &amp; Illustration': 'gnr_Design & Illustration'}, inplace=True)

df_games.columns

Index(['app_name', 'game_id', 'developer', 'release_year', 'gnr_Accounting',
       'gnr_Action', 'gnr_Adventure', 'gnr_Animation & Modeling',
       'gnr_Audio Production', 'gnr_Casual', 'gnr_Design & Illustration',
       'gnr_Early Access', 'gnr_Education', 'gnr_Free to Play', 'gnr_Indie',
       'gnr_Massively Multiplayer', 'gnr_Photo Editing', 'gnr_RPG',
       'gnr_Racing', 'gnr_Simulation', 'gnr_Software Training', 'gnr_Sports',
       'gnr_Strategy', 'gnr_Utilities', 'gnr_Video Production',
       'gnr_Web Publishing'],
      dtype='object')

Se presenta la información de nuestro dataframe hasta este punto, observando que las columnas de los géneros se crearon correctamente, sin valores nulos y conteniendo como valores posibles 0 o 1.

In [38]:
pd.set_option('display.max_columns', None)
df_games.head(1)

,app_name,game_id,developer,release_year,gnr_Accounting,gnr_Action,gnr_Adventure,gnr_Animation & Modeling,gnr_Audio Production,gnr_Casual,gnr_Design & Illustration,gnr_Early Access,gnr_Education,gnr_Free to Play,gnr_Indie,gnr_Massively Multiplayer,gnr_Photo Editing,gnr_RPG,gnr_Racing,gnr_Simulation,gnr_Software Training,gnr_Sports,gnr_Strategy,gnr_Utilities,gnr_Video Production,gnr_Web Publishing
88310,Lost Summoner Kitty,761140,Kotoshiro,2018,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0


In [39]:
my_functions.describe_df(df_games)

Cantidad Registros:  32131
Cantidad Campos:  26


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,app_name,object,0,0.00,32131,100.00,32094,99.88
1,game_id,object,0,0.00,32131,100.00,32131,100.00
2,developer,object,3297,10.26,28834,89.74,10992,34.21
3,release_year,int64,0,0.00,32131,100.00,45,0.14
4,gnr_Accounting,int64,0,0.00,32131,100.00,2,0.01
5,gnr_Action,int64,0,0.00,32131,100.00,2,0.01
6,gnr_Adventure,int64,0,0.00,32131,100.00,2,0.01
7,gnr_Animation & Modeling,int64,0,0.00,32131,100.00,2,0.01
8,gnr_Audio Production,int64,0,0.00,32131,100.00,2,0.01
9,gnr_Casual,int64,0,0.00,32131,100.00,2,0.01


## Campo 'developer'

Existen 10992 valores para 'developer', es decir, desarrolladores de los juegos.
Es el único campo que aún presenta datos nulos. Se procederá a completarlos con el texto 'Sin Datos'.


In [40]:
# Reemplazar valores nulos en la columna "developer" con 'Sin Datos'
df_games['developer'].fillna('Sin Datos', inplace=True)

De está forma ya se cuenta con un dataset limpio, sobre el cual se realizará el EDA.

In [41]:
my_functions.describe_df(df_games)

Cantidad Registros:  32131
Cantidad Campos:  26


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,app_name,object,0,0.0,32131,100.0,32094,99.88
1,game_id,object,0,0.0,32131,100.0,32131,100.00
2,developer,object,0,0.0,32131,100.0,10993,34.21
3,release_year,int64,0,0.0,32131,100.0,45,0.14
4,gnr_Accounting,int64,0,0.0,32131,100.0,2,0.01
5,gnr_Action,int64,0,0.0,32131,100.0,2,0.01
6,gnr_Adventure,int64,0,0.0,32131,100.0,2,0.01
7,gnr_Animation & Modeling,int64,0,0.0,32131,100.0,2,0.01
8,gnr_Audio Production,int64,0,0.0,32131,100.0,2,0.01
9,gnr_Casual,int64,0,0.0,32131,100.0,2,0.01


## 'steam_games_v1.csv'

Se guarda el dataframe generado en un nuevo archivo, que se utilizará en próximos pasos del proyecto.

In [42]:
df_games.to_csv('Datasets/Datasets ETL/steam_games_v1.csv', index=False, encoding='utf-8')